# Autoredenomination in localnet

## Overview

There is a mechanism in place where Arweave may redenominate its token: every balance, price, and reward
is multiplied by 1000 once the circulating supply crosses a protocol-defined threshold.

This notebook drives a full redenomination cycle on a running localnet node and validates
that every observable quantity transitions correctly:

1. **Setup** – connect to the node, compile record accessors, define HTTP and utility helpers.
2. **Pre-redenomination** – mine past the pricing transition, load the mining wallet,
   and override redenomination parameters so the cycle triggers quickly.
3. **Trigger** – submit transactions to push the reward pool past the
   threshold, mine the trigger block, and assert block reward and endowment pool updates.
4. **Redenomination** – mine through the scheduled redenomination height, verify the
   denomination increments, all HTTP pricing/wallet endpoints scale by 1000×, and
   block rewards and endowment pool updates follow the expected formula across the boundary.
5. **Post-redenomination** – validate every HTTP endpoint independently at the new
   denomination.
6. **Cleanup** – restore overridden parameters.

## Setup

### Connect to the localnet node

Starts a distributed Erlang node with long names, sets the cookie to `localnet`, and pings `main-localnet@127.0.0.1` to confirm connectivity.

In [1]:
Cookie = 'localnet',
Node = 'main-localnet@127.0.0.1',

HostHasDot =
	case string:split(atom_to_list(node()), "@") of
		[_Name, Host] ->
			case string:find(Host, ".") of
				nomatch ->
					false;
				_ ->
					true
			end;
		_ ->
			false
	end,

_ =
	case {node(), HostHasDot} of
		{nonode@nohost, _} ->
			net_kernel:start([list_to_atom("redenom_notebook@127.0.0.1"), longnames]);
		{_, true} ->
			ok;
		{_, false} ->
			net_kernel:stop(),
			net_kernel:start([list_to_atom("redenom_notebook@127.0.0.1"), longnames])
	end,

erlang:set_cookie(node(), Cookie).

true


In [2]:
{Node, net_adm:ping(Node)}.

{'main-localnet@127.0.0.1',pong}


### RPC and mining helpers

- `RPCCall(M, F, A)` -- calls `M:F(A)` on the remote node.
- `RPCHeight()` -- returns the current block height.
- `MineUntilHeight(H)` -- asks localnet to mine up to height H and polls until the node reaches it.
- `RPCBlockHashByHeight(H)`, `RPCBlockByHeight(H)` -- fetch block hash/record from the remote node's storage.

In [3]:
RPCCall = fun(M, F, A) -> rpc:call(Node, M, F, A) end,
RPCHeight = fun() -> RPCCall(ar_node, get_height, []) end,

WaitForHeight =
	fun
		(_, _TargetHeight, 0) ->
			{error, mine_until_height_timeout};
		(Self, TargetHeight, AttemptsLeft) ->
			case RPCHeight() >= TargetHeight of
				true ->
					ok;
				false ->
					timer:sleep(100),
					Self(Self, TargetHeight, AttemptsLeft - 1)
			end
	end,

MineUntilHeight =
	fun(TargetHeight) ->
		MineResult = RPCCall(ar_localnet, mine_until_height, [TargetHeight]),
		ok =
			case MineResult of
				ok ->
					ok;
				[] ->
					ok;
				Other ->
					{error, {unexpected_mine_until_height_result, Other}}
			end,
		WaitForHeight(WaitForHeight, TargetHeight, 1200)
	end.

#Fun<erl_eval.42.105768164>


In [4]:
RPCBlockHashByHeight =
	fun(Height) ->
		RPCCall(ar_block_index, get_element_by_height, [Height])
	end,

RPCBlockByHeight =
	fun(Height) ->
		Hash = RPCBlockHashByHeight(Height),
		RPCCall(ar_storage, read_block, [Hash])
	end.

#Fun<erl_eval.42.105768164>


### Record accessor modules

Compiles accessor modules at runtime so the notebook can read Erlang record fields:
- `nb_block` -- accessors for `#block{}` fields (height, denomination, reward_pool, reward, etc.).
- `nb_config` -- accessor for `#config.mining_addr`.
- `nb_tx` -- accessors for `#tx{}` fields (reward, denomination, id).
- `nb_pricing` -- exposes `?TOTAL_SUPPLY`, `?GiB`, and computes miner/endowment fee shares using `?MINER_FEE_SHARE`.

In [5]:
TmpDir = ".tmp/notebooks/",
ok = filelib:ensure_dir(filename:join([TmpDir, "keep"])),

CompileModule =
	fun(Name, Source) ->
		Path = filename:join([TmpDir, Name ++ ".erl"]),
		ok = file:write_file(Path, Source),
		{ok, Module, Bin} = compile:file(Path, [binary]),
		{module, Module} = code:load_binary(Module, Path, Bin)
	end,

BlockAccessors =
	lists:flatten([
		"-module(nb_block).\n",
		"-export([height/1, denomination/1, redenomination_height/1, reward_pool/1,\n",
		"         reward/1, wallet_list/1, txs/1, weave_size/1, reward_addr/1,\n",
		"         debt_supply/1, price_per_gib_minute/1, reward_history/1,\n",
		"         kryder_plus_rate_multiplier/1, kryder_plus_rate_multiplier_latch/1]).\n",
		"-include_lib(\"arweave/include/ar.hrl\").\n",
		"height(B) -> B#block.height.\n",
		"denomination(B) -> B#block.denomination.\n",
		"redenomination_height(B) -> B#block.redenomination_height.\n",
		"reward_pool(B) -> B#block.reward_pool.\n",
		"reward(B) -> B#block.reward.\n",
		"wallet_list(B) -> B#block.wallet_list.\n",
		"txs(B) -> B#block.txs.\n",
		"weave_size(B) -> B#block.weave_size.\n",
		"reward_addr(B) -> B#block.reward_addr.\n",
		"debt_supply(B) -> B#block.debt_supply.\n",
		"price_per_gib_minute(B) -> B#block.price_per_gib_minute.\n",
		"reward_history(B) -> B#block.reward_history.\n",
		"kryder_plus_rate_multiplier(B) -> B#block.kryder_plus_rate_multiplier.\n",
		"kryder_plus_rate_multiplier_latch(B) -> B#block.kryder_plus_rate_multiplier_latch.\n"
	]),

ConfigAccessors =
	lists:flatten([
		"-module(nb_config).\n",
		"-export([mining_addr/1]).\n",
		"-include_lib(\"arweave_config/include/arweave_config.hrl\").\n",
		"mining_addr(C) -> C#config.mining_addr.\n"
	]),

TXAccessors =
	lists:flatten([
		"-module(nb_tx).\n",
		"-export([reward/1, denomination/1, id/1]).\n",
		"-include_lib(\"arweave/include/ar.hrl\").\n",
		"reward(TX) -> TX#tx.reward.\n",
		"denomination(TX) -> TX#tx.denomination.\n",
		"id(TX) -> TX#tx.id.\n"
	]),

PricingAccessors =
	lists:flatten([
		"-module(nb_pricing).\n",
		"-export([total_supply/0, miner_fee_share/1, endowment_fee_share/1, gib/0]).\n",
		"-include_lib(\"arweave/include/ar.hrl\").\n",
		"-include_lib(\"arweave/include/ar_pricing.hrl\").\n",
		"total_supply() -> ?TOTAL_SUPPLY.\n",
		"gib() -> ?GiB.\n",
		"miner_fee_share(TXFee) ->\n",
		"\t{Dividend, Divisor} = ?MINER_FEE_SHARE,\n",
		"\tTXFee * Dividend div Divisor.\n",
		"endowment_fee_share(TXFee) ->\n",
		"\tTXFee - miner_fee_share(TXFee).\n"
	]),

CompileModule("nb_block", BlockAccessors),
CompileModule("nb_config", ConfigAccessors),
CompileModule("nb_tx", TXAccessors),
CompileModule("nb_pricing", PricingAccessors),
ok.

ok


### HTTP helpers

Defines HTTP helpers for the localnet node:
- `HTTPGet(URL)` – GET request returning `{ok, Body}` or `{error, Reason}`.
- `HTTPPostJSON(URL, Body)` – POST with JSON content type.
- `HTTPGetInteger(URL)` – GET, parse body as a non-negative integer.
- `HTTPGetJSONFee(URL)` – GET, parse the `"fee"` field from a JSON response.
- `HTTPGetJSONMap(URL)` – GET, parse body as a JSON map.
- `HTTPGetJSONList(URL)` – GET, parse body as a JSON list.
- `HTTPAssertNonEmpty(URL)` – GET, assert body is non-empty.
- `HTTPAssertBase64Bytes(URL, ExpectedSize)` – GET, decode base64 body and assert byte size.
- `HTTPGetBlockHash(Height)` – GET block JSON by height, extract `indep_hash`.

In [6]:
{ok, _} = application:ensure_all_started(inets),

LocalnetHTTPHost =
	case os:getenv("LOCALNET_HTTP_HOST") of
		false ->
			"127.0.0.1";
		Value ->
			Value
	end,

LocalnetHTTPPort =
	case os:getenv("LOCALNET_HTTP_PORT") of
		false ->
			"1984";
		Value ->
			Value
	end,

LocalnetNetworkName =
	case os:getenv("LOCALNET_HTTP_NETWORK") of
		false ->
			case os:getenv("LOCALNET_NETWORK_NAME") of
				false ->
					"arweave.localnet";
				Value2 ->
					Value2
			end;
		Value ->
			Value
	end,

BaseUrl = "http://" ++ LocalnetHTTPHost ++ ":" ++ LocalnetHTTPPort,

HTTPGet =
	fun(Url) ->
		Headers = [{"x-network", LocalnetNetworkName}],
		case httpc:request(get, {Url, Headers}, [], []) of
			{ok, {{_, 200, _}, _RespHeaders, Body}} ->
				{ok, Body};
			{ok, {{_, Status, _}, _RespHeaders, Body}} ->
				{error, {http_status, Status, Body}};
			Error ->
				{error, Error}
		end
	end,

HTTPPostJSON =
	fun(Url, Body) ->
		Headers = [{"content-type", "application/json"}, {"x-network", LocalnetNetworkName}],
		case httpc:request(post, {Url, Headers, "application/json", Body}, [], []) of
			{ok, {{_, 200, _}, _RespHeaders, RespBody}} ->
				{ok, RespBody};
			{ok, {{_, Status, _}, _RespHeaders, RespBody}} ->
				{error, {http_status, Status, RespBody}};
			Error ->
				{error, Error}
		end
	end,

HTTPGetInteger =
	fun(Url) ->
		case HTTPGet(Url) of
			{ok, Body} ->
				Bin = iolist_to_binary(Body),
				case catch binary_to_integer(Bin) of
					{'EXIT', _} ->
						{error, {not_integer, Bin}};
					Value ->
						case Value >= 0 of
							true ->
								Value;
							false ->
								{error, {negative_integer, Value}}
						end
				end;
			{error, Reason} ->
				{error, Reason}
		end
	end,

HTTPGetJSONFee =
	fun(Url) ->
		case HTTPGet(Url) of
			{ok, Body} ->
				Map = jiffy:decode(Body, [return_maps]),
				case maps:get(<<"fee">>, Map, undefined) of
					undefined ->
						{error, {missing_fee, Map}};
					FeeBin ->
						case catch binary_to_integer(FeeBin) of
							{'EXIT', _} ->
								{error, {not_integer, FeeBin}};
							Value ->
								case Value >= 0 of
									true ->
										Value;
									false ->
										{error, {negative_integer, Value}}
								end
						end
				end;
			{error, Reason} ->
				{error, Reason}
		end
	end,

HTTPGetJSONMap =
	fun(Url) ->
		case HTTPGet(Url) of
			{ok, Body} ->
				case jiffy:decode(Body, [return_maps]) of
					Map when is_map(Map) ->
						Map;
					Other ->
						{error, {not_map, Other}}
				end;
			{error, Reason} ->
				{error, Reason}
		end
	end,

HTTPGetJSONList =
	fun(Url) ->
		case HTTPGet(Url) of
			{ok, Body} ->
				case jiffy:decode(Body) of
					List when is_list(List) ->
						List;
					Other ->
						{error, {not_list, Other}}
				end;
			{error, Reason} ->
				{error, Reason}
		end
	end,

HTTPAssertNonEmpty =
	fun(Url) ->
		case HTTPGet(Url) of
			{ok, Body} ->
				case byte_size(iolist_to_binary(Body)) > 0 of
					true ->
						ok;
					false ->
						{error, {empty_body, Url}}
				end;
			{error, Reason} ->
				{error, Reason}
		end
	end,

HTTPAssertBase64Bytes =
	fun(Url, ExpectedSize) ->
		case HTTPGet(Url) of
			{ok, Body} ->
				Bin = iolist_to_binary(Body),
				case catch ar_util:decode(Bin) of
					{'EXIT', _} ->
						{error, {invalid_base64, Bin}};
					Decoded ->
						case byte_size(Decoded) == ExpectedSize of
							true ->
								ok;
							false ->
								{error, {unexpected_size, byte_size(Decoded), ExpectedSize}}
						end
				end;
			{error, Reason} ->
				{error, Reason}
		end
	end,

HTTPGetBlockHash =
	fun(Height) ->
		Url = BaseUrl ++ "/block/height/" ++ integer_to_list(Height),
		Map = HTTPGetJSONMap(Url),
		case maps:get(<<"indep_hash">>, Map, undefined) of
			undefined ->
				{error, {missing_indep_hash, Height, Map}};
			Hash when is_binary(Hash) ->
				Hash;
			Hash ->
				iolist_to_binary(Hash)
		end
	end,

ok.

ok


### Utility helpers

- `DecodeBase64(Value)` – decodes a base64url-encoded value, returning `{ok, Decoded}` or `{error, Reason}`.

In [7]:
DecodeBase64 =
	fun(Value) ->
		Bin = iolist_to_binary(Value),
		case catch ar_util:decode(Bin) of
			{'EXIT', _} ->
				{error, {invalid_base64, Bin}};
			Decoded ->
				{ok, Decoded}
		end
	end.

#Fun<erl_eval.42.105768164>


## Pre-redenomination

### Read chain state

Reads the current block height and displays the starting denomination, redenomination_height, reward_pool, and debt_supply.

In [8]:
Height0 = RPCHeight(),
Block0 = RPCBlockByHeight(Height0),
#{ height => nb_block:height(Block0),
  denomination => nb_block:denomination(Block0),
  redenomination_height => nb_block:redenomination_height(Block0),
  reward_pool => nb_block:reward_pool(Block0),
  debt_supply => nb_block:debt_supply(Block0) }.

#{height => 2069888,denomination => 4,redenomination_height => 2069886,
  reward_pool => 292873436146306634442575000,debt_supply => 0}


### Load mining address and wallet key

Reads the mining address from the node's config and loads the corresponding wallet key pair via RPC. The wallet is used to sign transactions later.

In [9]:
{ok, Config} = RPCCall(arweave_config, get_env, []),
MiningAddr = nb_config:mining_addr(Config),

#{mining_addr => ar_util:encode(MiningAddr)}.

#{mining_addr => <<"yjrOLPSHP1cvZ_y1bSLkLd8lIWhu9dsbSufXm9-QjsY">>}


In [10]:
MiningWallet = RPCCall(ar_wallet, load_key, [MiningAddr]).

{{{ecdsa,secp256k1},
  <<157,27,57,209,187,167,148,55,118,249,37,206,94,220,143,242,208,115,56,154,
    175,32,127,73,136,71,4,177,87,89,241,31>>,
  <<2,215,4,40,91,61,226,129,118,138,18,7,154,221,140,229,244,16,43,144,42,9,
    26,114,36,102,67,23,84,139,196,122,22>>},
 {{ecdsa,secp256k1},
  <<2,215,4,40,91,61,226,129,118,138,18,7,154,221,140,229,244,16,43,144,42,9,
    26,114,36,102,67,23,84,139,196,122,22>>}}


### Override redenomination parameters

Saves the current values of `redenomination_threshold`, `redenomination_delay_blocks`, and `locked_rewards_blocks`, then overrides them on the remote node:
- `redenomination_delay_blocks = 2` (redenomination fires 2 blocks after scheduling).
- `locked_rewards_blocks = 1` (rewards unlock after 1 block).
- The redenomination threshold is set later, after mining reveals the available supply.

Also defines local helpers `Pow1000Local` (computes 1000^N) and `RedenominateLocal` (scales amounts between denominations).

In [11]:
PrevRedenomThreshold = RPCCall(application, get_env, [arweave, redenomination_threshold]),
PrevRedenomDelay = RPCCall(application, get_env, [arweave, redenomination_delay_blocks]),
PrevLockedRewards = RPCCall(application, get_env, [arweave, locked_rewards_blocks]),

Pow1000Local =
	fun
		Pow(0) ->
			1;
		Pow(N) when N > 0 ->
			1000 * Pow(N - 1)
	end,

RedenominateLocal =
	fun(Amount, FromDenom, ToDenom) ->
		case ToDenom >= FromDenom of
			true ->
				Amount * Pow1000Local(ToDenom - FromDenom);
			false ->
				Amount div Pow1000Local(FromDenom - ToDenom)
		end
	end,

PoolGrowthTarget = 1000000000,

ok = RPCCall(application, set_env, [arweave, redenomination_delay_blocks, 2]),
ok = RPCCall(application, set_env, [arweave, locked_rewards_blocks, 1]),

#{ pool_growth_target => PoolGrowthTarget,
  prev_threshold => PrevRedenomThreshold,
  prev_delay => PrevRedenomDelay,
  prev_locked_rewards => PrevLockedRewards }.

#{pool_growth_target => 1000000000,
  prev_threshold => {ok,65707142423137694720779001},
  prev_delay => {ok,2},
  prev_locked_rewards => {ok,1}}


### Wallet balance helpers

Defines `WalletBalanceFromBlock(Block, Addr)`, which reads the balance from the block's wallet tree via RPC and redenominates it to the block's denomination. Also defines `MinerBalanceAt(Height)` as a shorthand for the mining address.

In [12]:
Pow1000 = Pow1000Local,
Redenominate = RedenominateLocal,

WalletBalanceFromBlock =
	fun(Block, Addr) ->
		Root = nb_block:wallet_list(Block),
		BlockDenom = nb_block:denomination(Block),
		AccountsMap = RPCCall(ar_wallets, get, [Root, Addr]),
		case maps:get(Addr, AccountsMap, not_found) of
			not_found ->
				0;
			{Balance, _LastTX} ->
				Redenominate(Balance, 1, BlockDenom);
			{Balance, _LastTX, AccountDenom, _Perm} ->
				Redenominate(Balance, AccountDenom, BlockDenom)
		end
	end,

MinerBalanceAt =
	fun(Height) ->
		Block = RPCBlockByHeight(Height),
		WalletBalanceFromBlock(Block, MiningAddr)
	end.

#Fun<erl_eval.42.105768164>


### Mine past the pricing transition and unlock the first reward

Mines `max(3, TransitionEnd - StartHeight)` blocks to ensure the 2.7.2 pricing transition is complete and at least one reward has been unlocked (with `locked_rewards_blocks = 1`, a reward earned at height H is applied at H+1).

**Assertions:**
- Miner balance is positive after mining.
- Sets the redenomination threshold to `AvailableSupply + 1` (where `AvailableSupply = TotalSupply * 1000^denomination + DebtSupply - RewardPool`), so any further endowment fee will push the circulating supply past the threshold and trigger redenomination.

In [13]:
StartHeight = RPCHeight(),
TransitionEnd =
	RPCCall(ar_pricing_transition, transition_start_2_7_2, []) +
		RPCCall(ar_pricing_transition, transition_length_2_7_2, []),
TargetHeight = erlang:max(StartHeight + 3, TransitionEnd),
ok = MineUntilHeight(TargetHeight),
{StartHeight, TargetHeight, TransitionEnd}.

{2069888,2069891,2069870}


In [14]:
BalanceAfterUnlock = MinerBalanceAt(TargetHeight),
ok = case BalanceAfterUnlock > 0 of
	true ->
		ok;
	false ->
		{error, {mining_balance_not_unlocked, BalanceAfterUnlock}}
end.

ok


In [15]:
BlockAfterTransition = RPCBlockByHeight(RPCHeight()),
DenomAfterTransition = nb_block:denomination(BlockAfterTransition),
TotalSupplyAfterTransition = RedenominateLocal(nb_pricing:total_supply(), 1, DenomAfterTransition),
RewardPoolAfterTransition = nb_block:reward_pool(BlockAfterTransition),
DebtSupplyAfterTransition = nb_block:debt_supply(BlockAfterTransition),
AvailableSupplyAfterTransition =
	TotalSupplyAfterTransition + DebtSupplyAfterTransition - RewardPoolAfterTransition,
ThresholdAfterTransition = AvailableSupplyAfterTransition + 1,
ok = RPCCall(application, set_env, [arweave, redenomination_threshold, ThresholdAfterTransition]),

#{ available_supply => AvailableSupplyAfterTransition,
  threshold => ThresholdAfterTransition,
  pool_growth_target => PoolGrowthTarget }.

#{threshold => 65707142423133030322557425001,pool_growth_target => 1000000000,
  available_supply => 65707142423133030322557425000}


## Trigger redenomination

Builds and submits transactions to push the reward pool past the redenomination threshold. No blocks are mined yet; transactions go to the mempool and are mined in the next step.

### TX builder module

Defines `nb_tx_builder:build_tx/5` (compiled dynamically). Also defines helpers: `GetTXAnchor` (fetches anchor via HTTP), `BuildTX(Reward, Data)` (builds a format-2 TX with a given reward and data payload), and `PostTX(TX)` (serializes and POSTs a transaction).

In [16]:
TXBuilder =
	lists:flatten([
		"-module(nb_tx_builder).\n",
		"-export([build_tx/5]).\n",
		"-include_lib(\"arweave/include/ar.hrl\").\n",
		"build_tx(Reward, Data, Anchor, Denomination, Wallet) ->\n",
		"\tDataSize = byte_size(Data),\n",
		"\tDataRoot =\n",
		"\t\tcase DataSize > 0 of\n",
		"\t\t\ttrue ->\n",
		"\t\t\t\tTreeTX = ar_tx:generate_chunk_tree(#tx{ data = Data }),\n",
		"\t\t\t\tTreeTX#tx.data_root;\n",
		"\t\t\tfalse ->\n",
		"\t\t\t\t<<>>\n",
		"\t\tend,\n",
		"\tBaseTX = #tx{\n",
		"\t\tformat = 2,\n",
		"\t\tdata = Data,\n",
		"\t\tdata_size = DataSize,\n",
		"\t\tdata_root = DataRoot,\n",
		"\t\treward = Reward,\n",
		"\t\tlast_tx = Anchor,\n",
		"\t\ttarget = <<>>,\n",
		"\t\tquantity = 0,\n",
		"\t\tdenomination = Denomination\n",
		"\t},\n",
		"\tar_tx:sign(BaseTX, Wallet).\n"
	]),

CompileModule("nb_tx_builder", TXBuilder),
ok.

ok


In [17]:
GetTXAnchor =
	fun() ->
		case HTTPGet(BaseUrl ++ "/tx_anchor") of
			{ok, AnchorB64} ->
				ar_util:decode(iolist_to_binary(AnchorB64));
			{error, Reason} ->
				erlang:error({tx_anchor_failed, Reason})
		end
	end,

BuildTX =
	fun(Reward, Data) ->
		Anchor = GetTXAnchor(),
		Block = RPCBlockByHeight(RPCHeight()),
		Denom = nb_block:denomination(Block),
		nb_tx_builder:build_tx(Reward, Data, Anchor, Denom, MiningWallet)
	end,

PostTX =
	fun(TX) ->
		Body = ar_serialize:jsonify(ar_serialize:tx_to_json_struct(TX)),
		HTTPPostJSON(BaseUrl ++ "/tx", Body)
	end.

#Fun<erl_eval.42.105768164>


### Compute transaction count

No blocks mined. Queries the minimum fee for 4 bytes via `/price/4/{addr}`. Sets `RewardPerTX = max(100000000, MinFee)`. Splits each fee as `MinerShare = Fee div 21`, `EndowmentShare = Fee - MinerShare`. Computes `RequiredTXs = ceil(PoolGrowthTarget / EndowmentShare)` and caps at `MaxTXs = BalanceAfterUnlock div RewardPerTX`.

**Assertion:** `TXCount >= RequiredTXs` (the miner has enough balance to cover all needed transactions).

In [18]:
AddrB64 = binary_to_list(ar_util:encode(MiningAddr)),
MinFeeFor4Bytes = HTTPGetInteger(BaseUrl ++ "/price/4/" ++ AddrB64),
RewardPerTX = max(100000000, MinFeeFor4Bytes),
MinerSharePerTX = RewardPerTX div 21,
EndowmentShare = RewardPerTX - MinerSharePerTX,
RequiredTXs = (PoolGrowthTarget + EndowmentShare - 1) div EndowmentShare,
MaxTXs = BalanceAfterUnlock div RewardPerTX,
TXCount = erlang:min(RequiredTXs, MaxTXs),

ok = case TXCount >= RequiredTXs of
	true ->
		ok;
	false ->
		{error, {insufficient_balance, BalanceAfterUnlock, RequiredTXs, RewardPerTX}}
end,

#{ reward_per_tx => RewardPerTX,
  miner_share_per_tx => MinerSharePerTX,
  endowment_share => EndowmentShare,
  required_txs => RequiredTXs,
  tx_count => TXCount }.

#{reward_per_tx => 4897618132653043,miner_share_per_tx => 233219911078716,
  endowment_share => 4664398221574327,required_txs => 1,tx_count => 1}


### Submit transactions

Builds all transactions with unique data payloads and posts them via the HTTP `/tx` endpoint. Asserts all submissions succeed.

In [19]:
TX1 = BuildTX(RewardPerTX, <<"tx_1">>).

{tx,2,
    <<139,172,22,196,100,45,173,245,156,11,227,184,28,54,206,233,222,210,38,
      113,148,195,169,70,236,87,68,5,15,80,113,11>>,
    <<253,63,204,233,203,204,115,239,32,134,4,214,184,100,165,193,180,84,171,
      198,148,28,152,234,44,193,176,146,214,230,209,245,44,32,9,254,99,89,31,
      142,22,88,155,13,102,255,102,228>>,
    <<2,215,4,40,91,61,226,129,118,138,18,7,154,221,140,229,244,16,43,144,42,9,
      26,114,36,102,67,23,84,139,196,122,22>>,
    <<202,58,206,44,244,135,63,87,47,103,252,181,109,34,228,45,223,37,33,104,
      110,245,219,27,74,231,215,155,223,144,142,198>>,
    [],<<>>,0,<<"tx_1">>,4,[],
    <<80,150,207,185,223,249,233,156,209,143,234,81,171,110,46,139,135,104,51,
      164,66,97,241,245,151,19,123,99,218,110,6,65>>,
    <<229,118,71,208,177,55,47,223,163,169,232,69,227,132,137,19,98,177,244,99,
      221,219,19,234,232,69,153,146,134,186,52,8,93,109,153,251,116,125,75,153,
      170,0,66,188,202,114,255,42,43,180,4,28,225,108,251,151,22,187,148,7,145,
 

In [20]:
TXID = nb_tx:id(TX1).

<<139,172,22,196,100,45,173,245,156,11,227,184,28,54,206,233,222,210,38,113,
  148,195,169,70,236,87,68,5,15,80,113,11>>


In [21]:
RPCCall(ar_tx_db, get_error_codes, [TXID]).

not_found


In [22]:
RemainingTXs =
	[BuildTX(RewardPerTX, <<"tx_", (integer_to_binary(N))/binary>>)
		|| N <- lists:seq(2, TXCount)],
TXs = [TX1 | RemainingTXs],
Results = [PostTX(TX) || TX <- TXs],
ok = case lists:all(fun({ok, _}) -> true; (_) -> false end, Results) of
	true ->
		ok;
	false ->
		{error, {tx_submission_failures, [R || R <- Results, element(1, R) =/= ok]}}
end.

ok


### Mine the trigger block

Mines 1 block. The transactions submitted above are included in this block. Displays the block's reward_pool and reward.

In [23]:
RedenomTriggerBlockHeight = RPCHeight() + 1,
ok = MineUntilHeight(RedenomTriggerBlockHeight),
RedenomTriggerBlock = RPCBlockByHeight(RedenomTriggerBlockHeight),
#{ redenom_trigger_block_height => RedenomTriggerBlockHeight,
  reward_pool => nb_block:reward_pool(RedenomTriggerBlock),
  reward => nb_block:reward(RedenomTriggerBlock) }.

#{reward_pool => 292873436150971032664149327,reward => 123478299179911078716,
  redenom_trigger_block_height => 2069892}


### Assert reward pool and miner reward

Reads the trigger block and its transactions. Queries the inflation reward at this height
and the average block interval from the node. Computes expected reward and endowment pool
from the protocol's reward formula:

**Expected block reward** = `max(Inflation + MinerFeeShare, StorageCost)`, where:
- `Inflation = redenominate(ar_inflation:calculate(Height), 1, PrevDenomination)`
- `MinerFeeShare = sum(TXFee div 21)` over block transactions
- `StorageCost = N_REPLICATIONS * WeaveSize * PricePerGiBMinute * BlockInterval / (60 * GiB)`

**Expected endowment pool** = `PrevPool + EndowmentFeeShare - max(0, StorageCost - BaseReward)`

Also defines `ComputeExpectedRewardAndPool` helper reused later for redenomination assertions.

**Assertions:**
- `block.reward` matches the expected reward within 0.1%.
- `block.reward_pool` matches the expected pool within 0.1%.

The 0.1% tolerance accounts for `ar_inflation:calculate/1` being queried once
at the trigger block height and reused for nearby blocks. The inflation reward
decays with height but changes by less than 0.001% per block at mainnet heights,
so 0.1% is conservative.

In [24]:
EnsureTXs =
	fun(TXs) ->
		case TXs of
			[] -> [];
			_ ->
				case lists:any(fun(TX) -> is_binary(TX) end, TXs) of
					true ->
						[RPCCall(ar_storage, read_tx, [TXID]) || TXID <- TXs];
					false ->
						TXs
				end
		end
	end,

ComputeFeeShares =
	fun(BlockTXs, Denom) ->
		case BlockTXs of
			[] ->
				{0, 0};
			_ ->
				lists:foldl(
					fun(TX, {MinerAcc, EndowmentAcc}) ->
						TXFeeBase = nb_tx:reward(TX),
						TXDenom = nb_tx:denomination(TX),
						TXFee = RedenominateLocal(TXFeeBase, TXDenom, Denom),
						MinerShare = TXFee div 21,
						{MinerAcc + MinerShare, EndowmentAcc + (TXFee - MinerShare)}
					end,
					{0, 0},
					BlockTXs
				)
		end
	end,

PrevRedenomTriggerBlock = RPCBlockByHeight(RedenomTriggerBlockHeight - 1),
RefInflationBase = RPCCall(ar_inflation, calculate, [RedenomTriggerBlockHeight]),
RefBlockInterval = RPCCall(ar_block_time_history, compute_block_interval,
	[PrevRedenomTriggerBlock]),
NReplications = 20,
GiB = nb_pricing:gib(),

ComputeExpectedRewardAndPool =
	fun(Height, Block, PrevBlock, BlockTXs) ->
		PrevDenomLocal = nb_block:denomination(PrevBlock),
		BlockDenomLocal = nb_block:denomination(Block),
		InflationLocal = RedenominateLocal(RefInflationBase, 1, PrevDenomLocal),
		{MinerFeeShareLocal, EndowmentFeeShareLocal} =
			ComputeFeeShares(BlockTXs, PrevDenomLocal),
		WeaveSizeLocal = nb_block:weave_size(Block),
		PriceLocal = nb_block:price_per_gib_minute(PrevBlock),
		StorageCostLocal = NReplications * WeaveSizeLocal * PriceLocal
			* RefBlockInterval div (60 * GiB),
		BaseRewardLocal = InflationLocal + MinerFeeShareLocal,
		PrevPoolLocal = nb_block:reward_pool(PrevBlock),
		Pool2 = PrevPoolLocal + EndowmentFeeShareLocal,
		{ExpReward0, ExpPool0} =
			case BaseRewardLocal >= StorageCostLocal of
				true ->
					{BaseRewardLocal, Pool2};
				false ->
					TakeLocal = StorageCostLocal - BaseRewardLocal,
					case TakeLocal > Pool2 of
						true ->
							{StorageCostLocal, 0};
						false ->
							{StorageCostLocal, Pool2 - TakeLocal}
					end
			end,
		ExpReward = RedenominateLocal(ExpReward0, PrevDenomLocal, BlockDenomLocal),
		ExpPool = RedenominateLocal(ExpPool0, PrevDenomLocal, BlockDenomLocal),
		#{
			expected_reward => ExpReward,
			expected_pool => ExpPool,
			inflation => InflationLocal,
			storage_cost => StorageCostLocal,
			miner_fee_share => MinerFeeShareLocal,
			endowment_fee_share => EndowmentFeeShareLocal
		}
	end,

TriggerBlockTXs = EnsureTXs(nb_block:txs(RedenomTriggerBlock)),
TriggerExpected = ComputeExpectedRewardAndPool(
	RedenomTriggerBlockHeight, RedenomTriggerBlock,
	PrevRedenomTriggerBlock, TriggerBlockTXs),

ActualReward = nb_block:reward(RedenomTriggerBlock),
ActualPool = nb_block:reward_pool(RedenomTriggerBlock),
ExpectedReward = maps:get(expected_reward, TriggerExpected),
ExpectedPool = maps:get(expected_pool, TriggerExpected),

RewardTolerance = max(1, ExpectedReward div 1000),
ok = case abs(ActualReward - ExpectedReward) =< RewardTolerance of
	true ->
		ok;
	false ->
		{error, {reward_mismatch, ActualReward, ExpectedReward, RewardTolerance}}
end,

PoolTolerance = max(1, ExpectedPool div 1000),
ok = case abs(ActualPool - ExpectedPool) =< PoolTolerance of
	true ->
		ok;
	false ->
		{error, {pool_mismatch, ActualPool, ExpectedPool, PoolTolerance}}
end,

TriggerExpected#{
	actual_reward => ActualReward,
	actual_pool => ActualPool
}.

#{miner_fee_share => 233219911078716,endowment_fee_share => 4664398221574327,
  expected_reward => 123478299179911078716,
  expected_pool => 292873436150971032664149327,
  inflation => 123478065960000000000,storage_cost => 0,
  actual_reward => 123478299179911078716,
  actual_pool => 292873436150971032664149327}


## Redenomination

### Schedule redenomination

Mines 1 block. Because the redenomination threshold was set just above the circulating
supply, the node schedules redenomination at the next opportunity.

**Assertion:** The new block's `redenomination_height` is greater than the current height
(redenomination has been scheduled for a future block, with a 2-block delay as configured).

In [25]:
ScheduleStartHeight = RPCHeight(),
ScheduleStartBlock = RPCBlockByHeight(ScheduleStartHeight),
PrevRedenomHeight = nb_block:redenomination_height(ScheduleStartBlock),

ok = MineUntilHeight(ScheduleStartHeight + 1),
ScheduleBlock = RPCBlockByHeight(ScheduleStartHeight + 1),
NewRedenomHeight = nb_block:redenomination_height(ScheduleBlock),

ok = case NewRedenomHeight > ScheduleStartHeight of
	true ->
		ok;
	false ->
		{error, {redenomination_not_scheduled, PrevRedenomHeight, NewRedenomHeight}}
end,

#{ scheduled_height => NewRedenomHeight }.

#{scheduled_height => 2069893}


### Snapshot HTTP values before redenomination

Mines to `NewRedenomHeight` (the block just before redenomination fires). Captures all HTTP pricing and wallet values into a `PreHTTP` map for comparison after redenomination.

In [26]:
PreRedenomHeight = NewRedenomHeight,
ok = MineUntilHeight(PreRedenomHeight),

PreInfoMap = HTTPGetJSONMap(BaseUrl ++ "/info"),
PreInfoHeight =
	case maps:get(<<"height">>, PreInfoMap, undefined) of
		Height when is_integer(Height) ->
			Height;
		HeightBin when is_binary(HeightBin) ->
			binary_to_integer(HeightBin);
		Other ->
			erlang:error({unexpected_info_height, Other})
	end,
ok = case PreInfoHeight == PreRedenomHeight of
	true ->
		ok;
	false ->
		{error, {pre_height_mismatch, PreInfoHeight, PreRedenomHeight}}
end,

AddrB64 = binary_to_list(ar_util:encode(MiningAddr)),

PreHTTP = #
{
	price0 => HTTPGetInteger(BaseUrl ++ "/price/0"),
	price1g => HTTPGetInteger(BaseUrl ++ "/price/1000000000"),
	price0_addr => HTTPGetInteger(BaseUrl ++ "/price/0/" ++ AddrB64),
	price1g_addr => HTTPGetInteger(BaseUrl ++ "/price/1000000000/" ++ AddrB64),
	price2_0 => HTTPGetJSONFee(BaseUrl ++ "/price2/0"),
	price2_1g => HTTPGetJSONFee(BaseUrl ++ "/price2/1000000000"),
	price2_0_addr => HTTPGetJSONFee(BaseUrl ++ "/price2/0/" ++ AddrB64),
	price2_1g_addr => HTTPGetJSONFee(BaseUrl ++ "/price2/1000000000/" ++ AddrB64),
	opt0 => HTTPGetJSONFee(BaseUrl ++ "/optimistic_price/0"),
	opt1g => HTTPGetJSONFee(BaseUrl ++ "/optimistic_price/1000000000"),
	opt0_addr => HTTPGetJSONFee(BaseUrl ++ "/optimistic_price/0/" ++ AddrB64),
	opt1g_addr => HTTPGetJSONFee(BaseUrl ++ "/optimistic_price/1000000000/" ++ AddrB64),
	v2_0 => HTTPGetInteger(BaseUrl ++ "/v2price/0"),
	v2_1g => HTTPGetInteger(BaseUrl ++ "/v2price/1000000000"),
	v2_0_addr => HTTPGetInteger(BaseUrl ++ "/v2price/0/" ++ AddrB64),
	v2_1g_addr => HTTPGetInteger(BaseUrl ++ "/v2price/1000000000/" ++ AddrB64),
	wallet_balance => HTTPGetInteger(BaseUrl ++ "/wallet/" ++ AddrB64 ++ "/balance"),
	reserved_rewards => HTTPGetInteger(BaseUrl ++ "/wallet/" ++ AddrB64 ++ "/reserved_rewards_total")
},
PreHTTP.

#{price0 => 59246720252252,price1g => 18458446185029267217,
  price0_addr => 59246720252252,price1g_addr => 18458446185029267217,
  price2_0 => 59246720252252,price2_1g => 18458446185029267217,
  price2_0_addr => 59246720252252,price2_1g_addr => 18458446185029267217,
  opt0 => 59246720252252,opt1g => 18458446185029267217,
  opt0_addr => 59246720252252,opt1g_addr => 18458446185029267217,
  v2_0 => 59246720252252,v2_1g => 18458446185029267217,
  v2_0_addr => 59246720252252,v2_1g_addr => 18458446185029267217,
  wallet_balance => 3332960484717335850673,
  reserved_rewards => 123477740282000000000}


### Assert redenomination at scheduled height

Mines 1 block to `NewRedenomHeight + 1` (the redenomination block). Captures all HTTP
pricing and wallet values in `PostHTTP`.

**Assertions:**
- `block.denomination` incremented exactly once.
- `/info` height matches; network name unchanged.
- **Wallet balance (exact):** `Post == (Pre + PreBlockReward) * 1000`.
- **Pricing endpoints (approximate):** expected to scale by 1000× but one block of
  economic activity occurs between snapshots. Tolerance: `max(1000, Expected div 10000)`
  (0.01% or 1000 Winston, whichever is larger).

In [27]:
RedenomBlockHeight = NewRedenomHeight + 1,
ok = MineUntilHeight(RedenomBlockHeight),
PreRedenomBlock = RPCBlockByHeight(RedenomBlockHeight - 1),
RedenomBlock = RPCBlockByHeight(RedenomBlockHeight),
DenomBefore = nb_block:denomination(PreRedenomBlock),
DenomAt = nb_block:denomination(RedenomBlock),

ok = case DenomAt == DenomBefore + 1 of
	true ->
		ok;
	false ->
		{error, {denomination_not_incremented, DenomBefore, DenomAt}}
end,

PostInfoMap = HTTPGetJSONMap(BaseUrl ++ "/info"),
PostInfoHeight =
	case maps:get(<<"height">>, PostInfoMap, undefined) of
		PostH when is_integer(PostH) ->
			PostH;
		PostHBin when is_binary(PostHBin) ->
			binary_to_integer(PostHBin);
		Other2 ->
			erlang:error({unexpected_info_height, Other2})
	end,
ok = case PostInfoHeight == RedenomBlockHeight of
	true ->
		ok;
	false ->
		{error, {post_height_mismatch, PostInfoHeight, RedenomBlockHeight}}
end,

PreNetwork = maps:get(<<"network">>, PreInfoMap, undefined),
PostNetwork = maps:get(<<"network">>, PostInfoMap, undefined),
ok = case PreNetwork == PostNetwork of
	true ->
		ok;
	false ->
		{error, {network_changed, PreNetwork, PostNetwork}}
end,

PostHTTP = #
{
	price0 => HTTPGetInteger(BaseUrl ++ "/price/0"),
	price1g => HTTPGetInteger(BaseUrl ++ "/price/1000000000"),
	price0_addr => HTTPGetInteger(BaseUrl ++ "/price/0/" ++ AddrB64),
	price1g_addr => HTTPGetInteger(BaseUrl ++ "/price/1000000000/" ++ AddrB64),
	price2_0 => HTTPGetJSONFee(BaseUrl ++ "/price2/0"),
	price2_1g => HTTPGetJSONFee(BaseUrl ++ "/price2/1000000000"),
	price2_0_addr => HTTPGetJSONFee(BaseUrl ++ "/price2/0/" ++ AddrB64),
	price2_1g_addr => HTTPGetJSONFee(BaseUrl ++ "/price2/1000000000/" ++ AddrB64),
	opt0 => HTTPGetJSONFee(BaseUrl ++ "/optimistic_price/0"),
	opt1g => HTTPGetJSONFee(BaseUrl ++ "/optimistic_price/1000000000"),
	opt0_addr => HTTPGetJSONFee(BaseUrl ++ "/optimistic_price/0/" ++ AddrB64),
	opt1g_addr => HTTPGetJSONFee(BaseUrl ++ "/optimistic_price/1000000000/" ++ AddrB64),
	v2_0 => HTTPGetInteger(BaseUrl ++ "/v2price/0"),
	v2_1g => HTTPGetInteger(BaseUrl ++ "/v2price/1000000000"),
	v2_0_addr => HTTPGetInteger(BaseUrl ++ "/v2price/0/" ++ AddrB64),
	v2_1g_addr => HTTPGetInteger(BaseUrl ++ "/v2price/1000000000/" ++ AddrB64),
	wallet_balance => HTTPGetInteger(BaseUrl ++ "/wallet/" ++ AddrB64 ++ "/balance"),
	reserved_rewards => HTTPGetInteger(BaseUrl ++ "/wallet/" ++ AddrB64 ++ "/reserved_rewards_total")
},

ParseIntValue =
	fun(Value) ->
		case Value of
			Int when is_integer(Int) ->
				Int;
			Bin when is_binary(Bin) ->
				binary_to_integer(Bin);
			Other ->
				erlang:error({unexpected_integer_value, Other})
		end
	end,

PreRewardBlock = HTTPGetJSONMap(BaseUrl ++ "/block/height/" ++ integer_to_list(RedenomBlockHeight - 1)),
PreRewardValue = maps:get(<<"reward">>, PreRewardBlock, undefined),
PreReward =
	case PreRewardValue of
		undefined ->
			erlang:error({missing_reward, PreRewardBlock});
		_ ->
			ParseIntValue(PreRewardValue)
	end,
PreWalletBalance = maps:get(wallet_balance, PreHTTP),
PostWalletBalance = maps:get(wallet_balance, PostHTTP),
ExpectedPostWallet = (PreWalletBalance + PreReward) * 1000,
ok = case PostWalletBalance == ExpectedPostWallet of
	true ->
		ok;
	false ->
		{error, {wallet_balance_scale_mismatch, PreWalletBalance, PreReward, PostWalletBalance}}
end,

Scale = 1000,

CheckApproxScale =
	fun(Key) ->
		PreVal = maps:get(Key, PreHTTP),
		PostVal = maps:get(Key, PostHTTP),
		Expected = PreVal * Scale,
		Diff = abs(PostVal - Expected),
		MaxDiff = max(1000, Expected div 10000),
		case Diff =< MaxDiff of
			true ->
				ok;
			false ->
				erlang:error({redenom_approx_scale_mismatch, Key, PreVal, PostVal, Diff, MaxDiff})
		end
	end,

ApproxScaleKeys = [
	price0, price1g, price0_addr, price1g_addr,
	price2_0, price2_1g, price2_0_addr, price2_1g_addr,
	opt0, opt1g, opt0_addr, opt1g_addr,
	v2_0, v2_1g, v2_0_addr, v2_1g_addr,
	reserved_rewards
],
lists:foreach(CheckApproxScale, ApproxScaleKeys),

PostHTTP.

#{price0 => 59246720252252719,price1g => 18458446185029267217100,
  price0_addr => 59246720252252719,price1g_addr => 18458446185029267217100,
  price2_0 => 59246720252252719,price2_1g => 18458446185029267217100,
  price2_0_addr => 59246720252252719,
  price2_1g_addr => 18458446185029267217100,opt0 => 59246720252252719,
  opt1g => 18458446185029267217100,opt0_addr => 59246720252252719,
  opt1g_addr => 18458446185029267217100,v2_0 => 59246720252252719,
  v2_1g => 18458446185029267217100,v2_0_addr => 59246720252252719,
  v2_1g_addr => 18458446185029267217100,
  wallet_balance => 3456438224999335850673000,
  reserved_rewards => 123477414604000000000000}


### Assert block reward and endowment pool around redenomination

Mines 1 block to `RedenomBlockHeight + 1`. Uses `ComputeExpectedRewardAndPool` (defined earlier)
to verify the block reward and endowment pool at three heights around the redenomination
boundary: `RedenomBlockHeight - 1` (pre), `RedenomBlockHeight` (redenomination block),
and `RedenomBlockHeight + 1` (post).

**Assertions:**
- At each height, `block.reward` matches the expected reward within 0.1%.
- At each height, `block.reward_pool` matches the expected pool within 0.1%.

See the trigger block cell above for the tolerance justification.

In [28]:
ok = MineUntilHeight(RedenomBlockHeight + 1),

AssertRewardAndPool =
	fun(CheckHeight) ->
		Block = RPCBlockByHeight(CheckHeight),
		PrevBlock = RPCBlockByHeight(CheckHeight - 1),
		BlockTXs = EnsureTXs(nb_block:txs(Block)),
		Expected = ComputeExpectedRewardAndPool(CheckHeight, Block, PrevBlock, BlockTXs),
		ActReward = nb_block:reward(Block),
		ActPool = nb_block:reward_pool(Block),
		ExpReward = maps:get(expected_reward, Expected),
		ExpPool = maps:get(expected_pool, Expected),
		RTol = max(1, ExpReward div 1000),
		ok =
			case abs(ActReward - ExpReward) =< RTol of
				true ->
					ok;
				false ->
					{error, {reward_mismatch, CheckHeight, ActReward, ExpReward, RTol}}
			end,
		PTol = max(1, ExpPool div 1000),
		ok =
			case abs(ActPool - ExpPool) =< PTol of
				true ->
					ok;
				false ->
					{error, {pool_mismatch, CheckHeight, ActPool, ExpPool, PTol}}
			end,
		Expected#{
			height => CheckHeight,
			actual_reward => ActReward,
			actual_pool => ActPool
		}
	end,

PreResult = AssertRewardAndPool(RedenomBlockHeight - 1),
AtResult = AssertRewardAndPool(RedenomBlockHeight),
PostResult = AssertRewardAndPool(RedenomBlockHeight + 1),

#{ pre => PreResult, at => AtResult, post => PostResult }.


#{at =>
      #{height => 2069894,miner_fee_share => 0,endowment_fee_share => 0,
        expected_reward => 123478065960000000000000,
        expected_pool => 292873436150971032664149327000,
        inflation => 123478065960000000000,storage_cost => 0,
        actual_reward => 123477414604000000000000,
        actual_pool => 292873436150971032664149327000},
  pre =>
      #{height => 2069893,miner_fee_share => 0,endowment_fee_share => 0,
        expected_reward => 123478065960000000000,
        expected_pool => 292873436150971032664149327,
        inflation => 123478065960000000000,storage_cost => 0,
        actual_reward => 123477740282000000000,
        actual_pool => 292873436150971032664149327},
  post =>
      #{height => 2069895,miner_fee_share => 0,endowment_fee_share => 0,
        expected_reward => 123478065960000000000000,
        expected_pool => 292873436150971032664149327000,
        inflation => 123478065960000000000000,storage_cost => 0,
        actual_reward => 12347708

### Assert miner balance deltas around redenomination

No blocks mined. Checks the miner balance delta at `RedenomBlockHeight - 1` (pre-redenomination) and `RedenomBlockHeight` (redenomination block).

For each height H, the expected balance delta at H+1 is `Redenominate(block(H).reward, block(H).denomination, block(H+1).denomination)`. The actual delta is `MinerBalanceAt(H+1) - Redenominate(MinerBalanceAt(H), block(H).denomination, block(H+1).denomination)`. Asserts exact equality.

In [29]:
CheckMinerDelta =
	fun(RewardHeight) ->
		RewardBlock = RPCBlockByHeight(RewardHeight),
		ApplyBlock = RPCBlockByHeight(RewardHeight + 1),
		Reward = nb_block:reward(RewardBlock),
		RewardDenom = nb_block:denomination(RewardBlock),
		ApplyDenom = nb_block:denomination(ApplyBlock),
		ExpectedApplied = Redenominate(Reward, RewardDenom, ApplyDenom),
		BalanceBefore = MinerBalanceAt(RewardHeight),
		BalanceAfter = MinerBalanceAt(RewardHeight + 1),
		BalanceBeforeNormalized = Redenominate(BalanceBefore, RewardDenom, ApplyDenom),
		Delta = BalanceAfter - BalanceBeforeNormalized,
		case Delta == ExpectedApplied of
			true ->
				ok;
			false ->
				{error, {miner_balance_delta_mismatch, RewardHeight, Delta, ExpectedApplied}}
		end
	end,

ok = CheckMinerDelta(RedenomBlockHeight - 1),
ok = CheckMinerDelta(RedenomBlockHeight),

ok.

ok


### Per-height summary table

No blocks mined. Displays a summary map for every height from `RedenomTriggerBlockHeight - 2` through `RedenomBlockHeight + 1`, showing denomination, redenomination_height, reward_pool, and miner_balance. This is an informational cell with no assertions.

In [30]:
SummaryStart0 = RedenomTriggerBlockHeight - 2,
SummaryStart =
	case SummaryStart0 < 0 of
		true ->
			0;
		false ->
			SummaryStart0
	end,
SummaryEnd = RedenomBlockHeight + 1,
Heights = lists:seq(SummaryStart, SummaryEnd),
Summary =
	[begin
		Block = RPCBlockByHeight(Height),
		#{ height => Height,
		  denomination => nb_block:denomination(Block),
		  redenomination_height => nb_block:redenomination_height(Block),
		  reward_pool => nb_block:reward_pool(Block),
		  miner_balance => WalletBalanceFromBlock(Block, MiningAddr) }
	 end || Height <- Heights],
Summary.

[#{height => 2069890,denomination => 4,redenomination_height => 2069886,
   reward_pool => 292873436146306634442575000,
   miner_balance => 2962529974195557425000},
 #{height => 2069891,denomination => 4,redenomination_height => 2069886,
   reward_pool => 292873436146306634442575000,
   miner_balance => 3086008691515557425000},
 #{height => 2069892,denomination => 4,redenomination_height => 2069893,
   reward_pool => 292873436150971032664149327,
   miner_balance => 3209482185537424771957},
 #{height => 2069893,denomination => 4,redenomination_height => 2069893,
   reward_pool => 292873436150971032664149327,
   miner_balance => 3332960484717335850673},
 #{height => 2069894,denomination => 5,redenomination_height => 2069893,
   reward_pool => 292873436150971032664149327000,
   miner_balance => 3456438224999335850673000},
 #{height => 2069895,denomination => 5,redenomination_height => 2069893,
   reward_pool => 292873436150971032664149327000,
   miner_balance => 3579915639603335850673000}

## Post-redenomination HTTP endpoint checks

Validates every HTTP pricing, wallet, and block endpoint at the current height
(post-redenomination). Each cell fetches a value from the HTTP API and asserts a property:

- **`/price/{size}`**: price for 0 bytes and 1 GB. `price(1GB) > price(0)`.
- **`/price/{size}/{addr}`**: with the miner address, equals the no-address variant.
- **`/price2/` and `/optimistic_price/`**: JSON fee endpoints. `price2 == price`,
  `optimistic_price <= price`.
- **`/v2price/`**: positive, monotonic in data size, address variant equals no-address.
- **`/wallet/{addr}/balance`**: equals the per-block balance endpoint.
- **`/wallet/{addr}/last_tx`**: decodes to 32 bytes; the TX JSON has matching `id`
  and non-empty `signature`.
- **`/wallet/{addr}/reserved_rewards_total`**: equals the current block's reward
  (`locked_rewards_blocks = 1`, sole miner).
- **`/tx_anchor`**: decodes to a block hash from the last 10 blocks.
- **`/block/height/{h}`**: returned height matches the request.
- **`/info`**: network name matches localnet.
- **`/tx/pending`**: every entry is a valid 32-byte base64-encoded TX id.
- **Redenomination scaling:** all pricing and wallet endpoints are approximately
  1000× their pre-redenomination values (tolerance: 0.01% or 1000 base units).

In [31]:
AddrB64 = binary_to_list(ar_util:encode(MiningAddr)),
InfoMap = HTTPGetJSONMap(BaseUrl ++ "/info"),
CurrentHeight =
	case maps:get(<<"height">>, InfoMap, undefined) of
		CurH when is_integer(CurH) ->
			CurH;
		CurHBin when is_binary(CurHBin) ->
			binary_to_integer(CurHBin);
		Other ->
			erlang:error({unexpected_info_height, Other})
	end,
#{addr => AddrB64, height => CurrentHeight}. 

#{addr => "yjrOLPSHP1cvZ_y1bSLkLd8lIWhu9dsbSufXm9-QjsY",height => 2069895}


### Assert post-redenomination values are ~1000× pre-redenomination

Compares every pricing and wallet endpoint at the current post-redenomination height
against the `PreHTTP` snapshot captured before redenomination. Between the two snapshots,
2 blocks were mined (redenomination block + 1 post block), so values may drift slightly
due to normal economic activity. Tolerance: `max(1000, Expected div 10000)` (0.01% or
1000 base units, whichever is larger).

In [32]:
PostHTTPCheck = #
{
	price0 => HTTPGetInteger(BaseUrl ++ "/price/0"),
	price1g => HTTPGetInteger(BaseUrl ++ "/price/1000000000"),
	price0_addr => HTTPGetInteger(BaseUrl ++ "/price/0/" ++ AddrB64),
	price1g_addr => HTTPGetInteger(BaseUrl ++ "/price/1000000000/" ++ AddrB64),
	v2_0 => HTTPGetInteger(BaseUrl ++ "/v2price/0"),
	v2_1g => HTTPGetInteger(BaseUrl ++ "/v2price/1000000000"),
	v2_0_addr => HTTPGetInteger(BaseUrl ++ "/v2price/0/" ++ AddrB64),
	v2_1g_addr => HTTPGetInteger(BaseUrl ++ "/v2price/1000000000/" ++ AddrB64),
	wallet_balance => HTTPGetInteger(BaseUrl ++ "/wallet/" ++ AddrB64 ++ "/balance"),
	reserved_rewards => HTTPGetInteger(BaseUrl ++ "/wallet/" ++ AddrB64 ++ "/reserved_rewards_total")
},

PostHTTPScaleKeys = [
	price0, price1g, price0_addr, price1g_addr,
	v2_0, v2_1g, v2_0_addr, v2_1g_addr,
	reserved_rewards
],

lists:foreach(
	fun(Key) ->
		PreVal = maps:get(Key, PreHTTP),
		PostVal = maps:get(Key, PostHTTPCheck),
		Expected = PreVal * 1000,
		Diff = abs(PostVal - Expected),
		MaxDiff = max(1000, Expected div 10000),
		case Diff =< MaxDiff of
			true ->
				ok;
			false ->
				erlang:error({post_redenomination_scale_mismatch,
					Key, PreVal, PostVal, Expected, Diff, MaxDiff})
		end
	end,
	PostHTTPScaleKeys
),

ok.

ok


In [33]:
Price0 = HTTPGetInteger(BaseUrl ++ "/price/0"),
Price0.

59246720252252719


In [34]:
Price1G = HTTPGetInteger(BaseUrl ++ "/price/1000000000"),
ok = case Price1G > Price0 of
	true ->
		ok;
	false ->
		{error, {price_not_monotonic, Price0, Price1G}}
end.

ok


In [35]:
Price0Addr = HTTPGetInteger(BaseUrl ++ "/price/0/" ++ AddrB64),
ok = case Price0Addr == Price0 of
	true ->
		ok;
	false ->
		{error, {price_addr_mismatch, Price0Addr, Price0}}
end.

ok


In [36]:
Price1GAddr = HTTPGetInteger(BaseUrl ++ "/price/1000000000/" ++ AddrB64),
ok = case Price1GAddr == Price1G of
	true ->
		ok;
	false ->
		{error, {price_addr_mismatch, Price1GAddr, Price1G}}
end.

ok


In [37]:
Price2_0 = HTTPGetJSONFee(BaseUrl ++ "/price2/0"),
ok = case Price2_0 == Price0 of
	true ->
		ok;
	false ->
		{error, {price2_mismatch, Price2_0, Price0}}
end.

ok


In [38]:
Price2_1G = HTTPGetJSONFee(BaseUrl ++ "/price2/1000000000"),
ok = case Price2_1G == Price1G of
	true ->
		ok;
	false ->
		{error, {price2_mismatch, Price2_1G, Price1G}}
end.

ok


In [39]:
Price2_0Addr = HTTPGetJSONFee(BaseUrl ++ "/price2/0/" ++ AddrB64),
ok = case Price2_0Addr == Price0Addr of
	true ->
		ok;
	false ->
		{error, {price2_mismatch, Price2_0Addr, Price0Addr}}
end.

ok


In [40]:
Price2_1GAddr = HTTPGetJSONFee(BaseUrl ++ "/price2/1000000000/" ++ AddrB64),
ok = case Price2_1GAddr == Price1GAddr of
	true ->
		ok;
	false ->
		{error, {price2_mismatch, Price2_1GAddr, Price1GAddr}}
end.

ok


In [41]:
Opt0 = HTTPGetJSONFee(BaseUrl ++ "/optimistic_price/0"),
ok = case Opt0 =< Price0 of
	true ->
		ok;
	false ->
		{error, {optimistic_too_high, Opt0, Price0}}
end.

ok


In [42]:
Opt1G = HTTPGetJSONFee(BaseUrl ++ "/optimistic_price/1000000000"),
ok = case Opt1G =< Price1G of
	true ->
		ok;
	false ->
		{error, {optimistic_too_high, Opt1G, Price1G}}
end.

ok


In [43]:
Opt0Addr = HTTPGetJSONFee(BaseUrl ++ "/optimistic_price/0/" ++ AddrB64),
ok = case Opt0Addr =< Price0Addr of
	true ->
		ok;
	false ->
		{error, {optimistic_too_high, Opt0Addr, Price0Addr}}
end.

ok


In [44]:
Opt1GAddr = HTTPGetJSONFee(BaseUrl ++ "/optimistic_price/1000000000/" ++ AddrB64),
ok = case Opt1GAddr =< Price1GAddr of
	true ->
		ok;
	false ->
		{error, {optimistic_too_high, Opt1GAddr, Price1GAddr}}
end.

ok


In [45]:
V2_0 = HTTPGetInteger(BaseUrl ++ "/v2price/0"),
ok = case V2_0 > 0 of
	true ->
		ok;
	false ->
		{error, {v2price_not_positive, V2_0}}
end.

ok


In [46]:
V2_1G = HTTPGetInteger(BaseUrl ++ "/v2price/1000000000"),
ok = case V2_1G > V2_0 of
	true ->
		ok;
	false ->
		{error, {v2price_not_monotonic, V2_0, V2_1G}}
end.

ok


In [47]:
V2_0Addr = HTTPGetInteger(BaseUrl ++ "/v2price/0/" ++ AddrB64),
ok = case V2_0Addr == V2_0 of
	true ->
		ok;
	false ->
		{error, {v2price_addr_mismatch, V2_0Addr, V2_0}}
end.

ok


In [48]:
V2_1GAddr = HTTPGetInteger(BaseUrl ++ "/v2price/1000000000/" ++ AddrB64),
ok = case V2_1GAddr == V2_1G of
	true ->
		ok;
	false ->
		{error, {v2price_addr_mismatch, V2_1GAddr, V2_1G}}
end.

ok


In [49]:
WalletBalanceHTTP = HTTPGetInteger(BaseUrl ++ "/wallet/" ++ AddrB64 ++ "/balance"),
WalletBalanceAtHeight =
	HTTPGetInteger(
		BaseUrl ++ "/block/height/" ++ integer_to_list(CurrentHeight) ++ "/wallet/" ++ AddrB64 ++ "/balance"
	),
ok = case WalletBalanceHTTP == WalletBalanceAtHeight of
	true ->
		ok;
	false ->
		{error, {wallet_balance_mismatch, WalletBalanceHTTP, WalletBalanceAtHeight}}
end.

ok


In [50]:
LastTXB64 =
	case HTTPGet(BaseUrl ++ "/wallet/" ++ AddrB64 ++ "/last_tx") of
		{ok, LastTXBody} ->
			iolist_to_binary(LastTXBody);
		{error, LastTXErr} ->
			erlang:error({last_tx_failed, LastTXErr})
	end,
ok = case DecodeBase64(LastTXB64) of
	{ok, LastTXDecoded} ->
		case byte_size(LastTXDecoded) == 32 of
			true ->
				ok;
			false ->
				{error, {last_tx_size_mismatch, byte_size(LastTXDecoded)}}
		end;
	{error, LastTXDecErr} ->
		{error, LastTXDecErr}
end,
LastTXMap = HTTPGetJSONMap(BaseUrl ++ "/tx/" ++ binary_to_list(LastTXB64)),
ok = case maps:get(<<"id">>, LastTXMap, undefined) of
	undefined ->
		{error, {missing_tx_id, LastTXMap}};
	LastTXB64 ->
		ok;
	OtherId ->
		{error, {last_tx_id_mismatch, OtherId, LastTXB64}}
end,
ok = case maps:get(<<"signature">>, LastTXMap, undefined) of
	Sig when is_binary(Sig), byte_size(Sig) > 0 ->
		ok;
	OtherSig ->
		{error, {invalid_tx_signature, OtherSig}}
end.

ok


In [51]:
ReservedHTTP = HTTPGetInteger(BaseUrl ++ "/wallet/" ++ AddrB64 ++ "/reserved_rewards_total"),

BlockForReserved = RPCBlockByHeight(CurrentHeight),
ExpectedReserved = nb_block:reward(BlockForReserved),

ok = case ReservedHTTP == ExpectedReserved of
	true ->
		ok;
	false ->
		{error, {reserved_rewards_mismatch, ReservedHTTP, ExpectedReserved}}
end.

ok


In [52]:
AnchorB64 =
	case HTTPGet(BaseUrl ++ "/tx_anchor") of
		{ok, AnchorBody} ->
			iolist_to_binary(AnchorBody);
		{error, AnchorErr} ->
			erlang:error({tx_anchor_failed, AnchorErr})
	end,
AnchorBin =
	case DecodeBase64(AnchorB64) of
		{ok, AnchorDecoded} ->
			AnchorDecoded;
		{error, AnchorDecErr} ->
			erlang:error({tx_anchor_invalid, AnchorDecErr})
	end,
RecentStart0 = CurrentHeight - 10,
RecentStart =
	case RecentStart0 < 0 of
		true ->
			0;
		false ->
			RecentStart0
	end,
RecentHeights = lists:seq(RecentStart, CurrentHeight),
RecentHashes =
	[begin
		HashB64 = HTTPGetBlockHash(Height),
		case DecodeBase64(HashB64) of
			{ok, HashBin} ->
				HashBin;
			{error, HashErr} ->
				erlang:error({invalid_block_hash, Height, HashErr})
		end
	 end || Height <- RecentHeights],
ok = case lists:member(AnchorBin, RecentHashes) of
	true ->
		ok;
	false ->
		{error, {tx_anchor_not_recent, AnchorB64, RecentHeights}}
end.

ok


In [53]:
BlockMap = HTTPGetJSONMap(BaseUrl ++ "/block/height/" ++ integer_to_list(CurrentHeight)),
ok = case maps:get(<<"height">>, BlockMap, undefined) of
	CurrentHeight ->
		ok;
	HeightVal ->
		{error, {block_height_mismatch, HeightVal, CurrentHeight}}
end.

ok


In [54]:
ok = case maps:get(<<"network">>, InfoMap, undefined) of
	undefined ->
		{error, {missing_network, InfoMap}};
	Network ->
		case Network == list_to_binary(LocalnetNetworkName) of
			true ->
				ok;
			false ->
				{error, {network_mismatch, Network, LocalnetNetworkName}}
		end
end.

ok


In [55]:
Pending = HTTPGetJSONList(BaseUrl ++ "/tx/pending"),
ok = case lists:all(
	fun(PendingItem) ->
		case DecodeBase64(PendingItem) of
			{ok, PendingDec} ->
				byte_size(PendingDec) == 32;
			{error, _} ->
				false
		end
	end,
	Pending
) of
	true ->
		ok;
	false ->
		{error, {pending_invalid_entries, Pending}}
end.

ok


## Cleanup

### Restore overridden parameters

Restores `redenomination_threshold`, `redenomination_delay_blocks`, and `locked_rewards_blocks` to the values saved before overriding.

In [56]:
RestoreEnv =
	fun(Key, Prev) ->
		case Prev of
			{ok, Value} ->
				RPCCall(application, set_env, [arweave, Key, Value]);
			undefined ->
				RPCCall(application, unset_env, [arweave, Key]);
			_ ->
				RPCCall(application, unset_env, [arweave, Key])
		end
	end,

RestoreEnv(redenomination_threshold, PrevRedenomThreshold),
RestoreEnv(redenomination_delay_blocks, PrevRedenomDelay),
RestoreEnv(locked_rewards_blocks, PrevLockedRewards),

ok.

ok
